# Society of Minds with Nested Chats in AutoGen

This notebook showcases how to use conversable agents in AutoGen to realize a Society of Mind (SoM) scenario, in which an agent runs a  chat session with another agent as an inner-monologue. The idea of SoM and part of the example demonstrated in this notebook come from this PR: https://github.com/microsoft/autogen/pull/890
 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

## Society of Minds with Nested Chats

In [1]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 44}

financial_tasks = [
    """On which days this year was Microsoft Stock higher than $400? Put results in a table.""",
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Construct a group chat manager to be used for the inner-monologue

In [2]:
assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [3]:
financial_assistant_1 = autogen.AssistantAgent(
    name="Financial_assistant_1",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

financial_assistant_2 = autogen.AssistantAgent(
    name="Financial_assistant_2",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

financial_assistant_1.register_nested_chats(
    [autogen.Agent, None], [{"recipient": manager, "summary_method": "reflection_with_llm"}]
)
user.initiate_chat(financial_assistant_1, message=financial_tasks[0])

User (to Financial_assistant_1):

On which days this year was Microsoft Stock higher than $400? Put results in a table.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
On which days this year was Microsoft Stock higher than $400? Put results in a table.


With the following carryover: 


********************************************************************************
Financial_assistant_1 (to chat_manager):

On which days this year was Microsoft Stock higher than $400? Put results in a table.


--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

To find out on which days Microsoft Stock (MSFT) was higher than $400 in the current year, we would typically query a financial data API or use a library such as `pandas` with `pandas_datareader` to extract historical sto

ChatResult(chat_history=[{'content': 'On which days this year was Microsoft Stock higher than $400? Put results in a table.', 'role': 'assistant'}, {'content': 'Microsoft stock (MSFT) exceeded a closing price of $400 on the following dates in the year 2024:\n\n- January 24 ($402.56)\n- January 25 ($404.87)\n- January 26 ($403.93)\n- January 29 ($409.72)\n- January 30 ($408.59)\n- February 1 ($403.78)\n- February 2 ($411.22)\n- February 5 ($405.65)\n- February 6 ($405.49)\n- February 7 ($414.05)\n- February 8 ($414.11)\n\nThe dates include the time and timezone offset for the Eastern Time Zone.', 'role': 'user'}, {'content': '', 'role': 'assistant'}], summary='', cost=({'total_cost': 0}, {'total_cost': 0}), human_input=[])